In [1]:
def flag_owner_with_improvements(df, improvement, owner_name):

    improvement_list = (
        df[df[improvement].sum(axis=1) > 0][owner_name]
        .drop_duplicates()
        .reset_index(drop=True)
        .to_list()
    )

    df["owner_with_improvement"] = np.where(df[owner_name].isin(improvement_list), 1, 0)

    return df

In [ ]:
def remove_owner_name_blank(df, owner_name_col):

    df["owner_name_blank_removed"] = df[owner_name_col].replace(
        r"^\s*$", np.nan, regex=True
    )

    df = df[~df["owner_name_blank_removed"].isnull()]

    return df

In [2]:
def remove_owner_name_filter_items(df, owner_name, owner_name_final, owner_name_filter):

    owner_name_filter.append("\d+")

    df = df[
        ~df[[owner_name, owner_name_final]]
        .apply(
            lambda x: x.str.contains("|".join(owner_name_filter), case=False, na=False)
        )
        .any(axis=1)
    ].reset_index(drop=True)

    return df

In [19]:
def add_comma_to_name_column(x):

    if "," not in x:

        return re.sub(r"^([^\s]*)\s+", r"\1, ", x)

    else:
        return x

In [5]:
def clean_owner_name(df):

    df["chosen_owner_name_to_process"] = df["chosen_owner_name"].str.replace(
        "et al|et. al.|etal| the ", "", regex=True, case=False
    )

    df["chosen_owner_name_to_process"] = df["chosen_owner_name_to_process"].apply(
        lambda x: add_comma_to_name_column(x)
    )

    df["full_name"] = df.chosen_owner_name_to_process.apply(lambda s: HumanName(s))

    for index, row in df.iterrows():
        row["full_name"].capitalize()

    df["format_name_dict"] = df.full_name.astype(str).apply(
        lambda s: HumanName(s).as_dict()
    )
    df["first_name"] = df["format_name_dict"].apply(lambda x: x.get("first"))

    df["last_name"] = df["format_name_dict"].apply(lambda x: x.get("last"))

    return df